In [57]:
# Initiate game
# generate n_players suits where n_players is currently 5
# generate n_cards per suit
# shuffle cards


# Player
# Once the trade is executed, register the trade on their own ledger
## information recorded
# Functionalities reveal private information
# Compute private and perceived probabilities
# generate insights
# plot risks


# MarketMakers
# ability to set bid and ask price
# arbitage alert


In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Functions

In [59]:
def shuffle(deck):
    shuffled_deck = []
    for i in range(len(deck)):
        idx = np.random.randint(0, len(deck))
        shuffled_deck.append(deck.pop(idx))
    return shuffled_deck

In [60]:
def generate_cards(n_cards, suits):
    deck = []
    for suit in suits:
        cards = [suit + ' '+ str(num) for num in np.linspace(1, n_cards, n_cards).astype('int')]
        deck.extend(cards)
    return deck

In [61]:
def distribute(deck, n_cards):
    private_information = []
    for idx in range(n_cards):
        private_information.append(deck.pop(np.random.randint(0, len(deck))))
    return private_information
    

In [62]:
def mergePrivateAndPublic(privateCards, revealedCards):
    temp = privateCards.copy()
    temp.extend(revealedCards)
    return temp

# to calculated private prob
# first merge public and private

# to calculated public prob
# use public cards only

# Note once a private card has been revealed, it should be added to the revealed public card 
def calculateProbability(cardsExcluded, n_cards_per_suit, suits):
    d = {}
    for s in suits:
        d[s] = n_cards_per_suit
        
    for c in cardsExcluded:
        suit, _ = c.split(' ')
        d[suit] -= 1
    # convert to prob
    return [round(d[key]/n_cards_per_suit, 3) for key in suits] 
    

In [63]:
def executeTrade(seller, buyer, bidAskPrice, contract, numContracts, idx, action, trade):
    seller.sell(buyer, bidAskPrice, contract, numContracts, idx, action, trade)
    buyer.buy(seller, bidAskPrice, contract, numContracts, idx, action, trade)

# Classes

In [120]:
class PublicBoard:
    suits = ['S', 'H', 'C', 'D']
    n_cards_per_suit = 13
    def __init__(self):
        self.revealedCards = []
        d = {}
        for key in ['Public Prob ' + str(suit) for suit in suits]:
            d[key] = [1.0]
        self.publicInformation = pd.DataFrame.from_dict(d, orient='columns')
    
    def updateProbability(self, idx):
        # update public probability
        self.publicInformation.loc[idx] = calculateProbability(self.revealedCards, n_cards_per_suit, suits)
    
    def revealedFromPublicDeck(self, deck):
        revealedCard = deck.pop()
        self.revealedCards.append(revealedCard)
        return revealedCard
        
    # add plotting tools here
    # bid and ask price record per action

In [121]:
# player includes market makers and speculator
class Player:
    suits = ['S', 'H', 'C', 'D']
    n_cards_per_suit = 13
    def __init__(self, privateCards, name):
        self.name = name
        self.privateCards = privateCards
        
        self.ledger = pd.DataFrame(columns=['Action', 'CounterParty', 'Contract', 'Price', 'Quantity', 'Buy/Sell']+\
                                   suits+['Cash'])
        self.inventories = {'S':0, 'H':0, 'C':0, 'D':0}
        self.cash = 0

        
        d = {}
        for key in ['Private Prob ' + str(suit) for suit in suits]:
            d[key] = [1.0]
        self.privateInformation = pd.DataFrame.from_dict(d, orient='columns')
        
    def updateProbability(self, idx, revealedCards):
        # update private probability
        self.privateInformation.loc[idx] = calculateProbability(\
                                            mergePrivateAndPublic(self.privateCards, revealedCards), n_cards_per_suit, suits)
    
    def revealPrivateCard(self, cardToBeRevealed):
        if cardToBeRevealed in self.privateInfo:
            idx = self.privateInfo.idx(cardToBeRevealed)
            cardRevealed = self.privateCards.pop(idx)
            return cardRevealed
        else:
            raise Exception("Card not in private deck.")
#     bid is higher than ask, [bid, ask]
    def buy(self, counterParty, bidAskPrice, contract, numContracts, idx, action, trade):
        self.inventories[contract] += numContracts 
        tradeNum = float(str(action) +'.'+str(trade))
        if contract == self.name:
            self.cash -= numContracts*bidAskPrice[1]
            entry = [tradeNum, counterParty.name, contract, -1*bidAskPrice[1], numContracts, 'Buy'] + \
                                   [self.inventories[key] for key in suits] + \
                                   [self.cash]
        else:
            self.cash -= numContracts*bidAskPrice[0]
            entry = [tradeNum, counterParty.name, contract, -1*bidAskPrice[0], numContracts, 'Buy'] + \
                                   [self.inventories[key] for key in suits] + \
                                   [self.cash]
        # update ledger
        self.ledger.loc[idx] = entry
                    
    
    def sell(self, counterParty, bidAskPrice, contract, numContracts, idx, action, trade):
        self.inventories[contract] -= numContracts
        tradeNum = float(str(action) +'.'+str(trade))
        if contract == self.name:
            self.cash += numContracts*bidAskPrice[0]
            entry = [tradeNum, counterParty.name, contract, bidAskPrice[0], numContracts, 'Sell'] + \
                                   [self.inventories[key] for key in suits] + \
                                   [self.cash]
        else:
            self.cash += numContracts*bidAskPrice[1]
            entry = [tradeNum, counterParty.name, contract, bidAskPrice[1], numContracts, 'Sell'] + \
                                   [self.inventories[key] for key in suits] + \
                                   [self.cash]
        # update ledger
        self.ledger.loc[idx] = entry

        

# Initiate Game

In [122]:
# number of market makers
n_market_makers = 4 # this is always fixed
suits = ['S', 'C', 'D', 'H'] 
# number of spectator
n_spectator = 1 # either 0 or 1

# total number of players
n_total_players = n_market_makers + n_spectator

# number of cards per suit
n_cards_per_suit = 13
# n_total_cards = n_cards*n_market_makers

# maximum bid-ask spread
max_bid_ask_spread = 5


In [123]:
###### Inititate Game ######
# 1. generate cards
deck = generate_cards(n_cards_per_suit, suits)
print('{} Cards generated.'.format(len(deck)))

# 2. shuffle deck
deck = shuffle(deck)
print('Cards shuffled.')

# 3. distribute cards
if n_total_players == 4:
    # distribute 4 cards for MM
    privateInfoMM1, privateInfoMM1,\
    privateInfoMM1, privateInfoMM1 = [distribute(deck, 4) for _ in range(4)]
elif n_total_players == 5:
    # distribute 5 cards for MM
    privateInfoMM1, privateInfoMM2,\
    privateInfoMM3, privateInfoMM4 = [distribute(deck, 5) for _ in range(4)]
    
    # 3 cards for spectator
    privateInfoSpec = distribute(deck, 3)
print('Players obtained private information.')
print('{} Cards left'.format(len(deck)))

52 Cards generated.
Cards shuffled.
Players obtained private information.
29 Cards left


# 2. Create Players and Public Board
1. Created players and public boad
2. Simulated the action of revealing cards from the deck. Updated the public probabilities and the perceived probabilities simultaneously

TODO:
set bid ask price
execute trade: buy or sell
update inventories

In [180]:
# create players
p1 = Player(privateInfoMM1, suits[0])

p2 = Player(privateInfoMM2, suits[1])

p3 = Player(privateInfoMM3, suits[2])

p4 = Player(privateInfoMM4, suits[3])

p5 = Player(privateInfoSpec, 'Spectator')

# for convenience
players = [p1, p2, p3, p4 ,p5]

In [181]:
# create public board
pb = PublicBoard()

In [182]:
#  example of card reveal, copy deck to avoid regenerating deck of cards
tempDeck = deck.copy()
num = 0
print(len(tempDeck))

# reveal 10 cards consecutively
for _ in range(10):
    num += 1
    pb.revealedFromPublicDeck(tempDeck)
    print(pb.revealedCards)
    pb.updateProbability(num)
    for player in players:
        player.updateProbability(num, pb.revealedCards)
    
# display public information
pb.publicInformation
print(len(tempDeck))

29
['H 11']
['H 11', 'H 6']
['H 11', 'H 6', 'S 9']
['H 11', 'H 6', 'S 9', 'H 3']
['H 11', 'H 6', 'S 9', 'H 3', 'H 7']
['H 11', 'H 6', 'S 9', 'H 3', 'H 7', 'C 7']
['H 11', 'H 6', 'S 9', 'H 3', 'H 7', 'C 7', 'S 8']
['H 11', 'H 6', 'S 9', 'H 3', 'H 7', 'C 7', 'S 8', 'C 11']
['H 11', 'H 6', 'S 9', 'H 3', 'H 7', 'C 7', 'S 8', 'C 11', 'D 8']
['H 11', 'H 6', 'S 9', 'H 3', 'H 7', 'C 7', 'S 8', 'C 11', 'D 8', 'D 12']
19


In [183]:
for player in players:
    player.updateProbability(1, pb.revealedCards)
    print(player.privateCards)
    print('########## Public Info ##########')
    print(pb.publicInformation.loc[10])
    print('########## ' + player.name + ' Private Info ##########')
    print(player.privateInformation.loc[10])



['H 1', 'S 6', 'D 6', 'S 2', 'H 9']
########## Public Info ##########
Public Prob S    0.846
Public Prob C    0.846
Public Prob D    0.846
Public Prob H    0.692
Name: 10, dtype: float64
########## S Private Info ##########
Private Prob S    0.692
Private Prob C    0.846
Private Prob D    0.769
Private Prob H    0.538
Name: 10, dtype: float64
['H 5', 'C 3', 'D 11', 'D 3', 'S 4']
########## Public Info ##########
Public Prob S    0.846
Public Prob C    0.846
Public Prob D    0.846
Public Prob H    0.692
Name: 10, dtype: float64
########## C Private Info ##########
Private Prob S    0.769
Private Prob C    0.769
Private Prob D    0.692
Private Prob H    0.615
Name: 10, dtype: float64
['C 10', 'C 4', 'H 4', 'D 13', 'H 2']
########## Public Info ##########
Public Prob S    0.846
Public Prob C    0.846
Public Prob D    0.846
Public Prob H    0.692
Name: 10, dtype: float64
########## D Private Info ##########
Private Prob S    0.846
Private Prob C    0.692
Private Prob D    0.769
Private Pro

In [191]:
numContracts = np.random.randint(0, 5)
ask = np.random.randint(20, 35)
bid = np.random.randint(ask, ask + max_bid_ask_spread)
idx = 0
for action in range(1, 4):
    for trade in range(np.random.randint(1, 10)):
        idx += 1
        numContracts = np.random.randint(1, 5)
        ask = np.random.randint(20, 35)
        bid = np.random.randint(ask, ask + max_bid_ask_spread)
        p1Sell = np.random.randint(0, 2)
        if p1Sell:
            executeTrade(seller=p1, buyer=p2, bidAskPrice=[bid, ask], \
                     contract=suits[np.random.randint(0,2)],numContracts=numContracts,idx=idx,action=action,trade=trade)
        else:
            executeTrade(seller=p2, buyer=p1, bidAskPrice=[bid, ask], \
                     contract=suits[np.random.randint(0,2)],numContracts=numContracts,idx=idx,action=action,trade=trade)

In [193]:
# zero sum tests
# player buys/sells its own suit then the logic is correct
# player buys/sells other suits then the logic is also correct
p1.ledger

,Action,CounterParty,Contract,Price,Quantity,Buy/Sell,S,C,D,H,Cash
1,1.0,C,S,-32,1,Buy,-31,-5,0,0,982
2,1.1,C,C,29,1,Sell,-31,-6,0,0,1011
3,1.2,C,S,24,4,Sell,-35,-6,0,0,1107
4,1.3,C,S,-22,1,Buy,-34,-6,0,0,1085
5,2.0,C,C,-32,1,Buy,-34,-5,0,0,1053
6,2.1,C,S,-23,3,Buy,-31,-5,0,0,984
7,2.2,C,C,33,1,Sell,-31,-6,0,0,1017
8,2.3,C,S,32,2,Sell,-33,-6,0,0,1081
9,2.4,C,S,30,3,Sell,-36,-6,0,0,1171
10,3.0,C,C,20,1,Sell,-36,-7,0,0,1191


In [194]:
p2.ledger

,Action,CounterParty,Contract,Price,Quantity,Buy/Sell,S,C,D,H,Cash
1,1.0,S,S,32,1,Sell,31,5,0,0,-982
2,1.1,S,C,-29,1,Buy,31,6,0,0,-1011
3,1.2,S,S,-24,4,Buy,35,6,0,0,-1107
4,1.3,S,S,22,1,Sell,34,6,0,0,-1085
5,2.0,S,C,32,1,Sell,34,5,0,0,-1053
6,2.1,S,S,23,3,Sell,31,5,0,0,-984
7,2.2,S,C,-33,1,Buy,31,6,0,0,-1017
8,2.3,S,S,-32,2,Buy,33,6,0,0,-1081
9,2.4,S,S,-30,3,Buy,36,6,0,0,-1171
10,3.0,S,C,-20,1,Buy,36,7,0,0,-1191


In [144]:
# Player
# Once the trade is executed, register the trade on their own ledger
## information recorded
# Functionalities reveal private information
# Compute private and perceived probabilities
# generate insights
# plot risks